In [28]:
# Load required libraries
library(data.table)
library(tximport)
library(DESeq2)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(TxDb.Mmusculus.UCSC.mm10.knownGene)
library(apeglm)
library(ggplot2)
library(dplyr)

In [29]:
# Function to load sample metadata
load_samples <- function(samples_file) {
  fread(samples_file, stringsAsFactors = FALSE)
}

# Function to validate file paths
validate_files <- function(samples, salmon_path) {
  files <- file.path(salmon_path, samples$Run, "quant.sf")
  names(files) <- samples$Run
  if (!all(file.exists(files))) stop("Some files are missing!")
  files
}

# Function to load gene name mappings
load_gene_names <- function(gene_name_file) {
  fread(gene_name_file, stringsAsFactors = FALSE)
}

# Function to load transcript-to-gene mapping
load_tx2gene <- function(tx2gene_file) {
  fread(tx2gene_file, stringsAsFactors = FALSE)
}

# Function to import Salmon quantification
import_salmon <- function(files, tx2gene) {
  tximport(files, type = "salmon", tx2gene = tx2gene, ignoreAfterBar = TRUE)
}

# Function to save TPM matrix
save_tpm_matrix <- function(txi_salmon, output_path) {
  tpm_file <- file.path(output_path, "salmon_tpm_matrix.csv")
  write.csv(txi_salmon$abundance, tpm_file, row.names = TRUE)
}

#Function to prepare DESeq2 dataset
prepare_deseq2 <- function(txi_salmon, samples, ref_group) {
  sample_table <- data.frame(group = samples$group)
  rownames(sample_table) <- colnames(txi_salmon$counts)
  sample_table$group <- relevel(factor(sample_table$group), ref = ref_group)
  DESeqDataSetFromTximport(txi_salmon, sample_table, ~ group)
}

create_tpm_matrix <- function(samples_file, gene_name_file, salmon_path, tx2gene_file, output_path) {
  samples <- load_samples(samples_file)
  files <- validate_files(samples, salmon_path)
  tx2gene <- load_tx2gene(tx2gene_file)
  txi_salmon <- import_salmon(files, tx2gene)
  save_tpm_matrix(txi_salmon, output_path)
  
  # Return txi_salmon for later use
  return(txi_salmon)
}

# Function to perform PCA and save the plot with sample names for outliers
perform_pca <- function(dds, output_path, outlier_threshold = 2) {
  rld <- rlog(dds)  # Log transformation
  pca_data <- plotPCA(rld, intgroup = "group", returnData = TRUE)
  percentVar <- round(100 * attr(pca_data, "percentVar"))
  
  # Calculate distances from the centroid
  centroid <- colMeans(pca_data[, c("PC1", "PC2")])
  pca_data$distance <- sqrt((pca_data$PC1 - centroid[1])^2 + (pca_data$PC2 - centroid[2])^2)
  
  # Identify outliers
  outlier_samples <- pca_data[pca_data$distance > outlier_threshold, ]
  
  p <- ggplot(pca_data, aes(PC1, PC2, color = group, label = name)) +
    geom_point(size = 3) +
    xlab(paste0("PC1: ", percentVar[1], "% variance")) +
    ylab(paste0("PC2: ", percentVar[2], "% variance")) +
    theme_bw() +  # Set background to white
    ggtitle("PCA of Samples") +
    geom_text(data = outlier_samples, aes(label = name), nudge_y = 0.5, check_overlap = TRUE)
  
  pca_file <- file.path(output_path, "PCA_plot.png")
  ggsave(pca_file, p, width = 8, height = 6)
  
  # Save a list of outlier samples to a file
  outliers_file <- file.path(output_path, "outlier_samples.csv")
  write.csv(outlier_samples, outliers_file, row.names = FALSE)
}

# Function to process DESeq2 results
process_deseq2_results <- function(dds, gene_name, output_path) {
  for (i in seq_along(resultsNames(dds))[-1]) {
    result_name <- resultsNames(dds)[i]
    res <- results(dds, name = result_name)
    raw_res_file <- file.path(output_path, paste0(result_name, "_raw.csv"))
    write.csv(res, raw_res_file, row.names = TRUE)
    
    res_data <- fread(raw_res_file, stringsAsFactors = FALSE)
    res_data$V1 <- sapply(strsplit(as.character(res_data$V1), "\\."), function(x) x[1])
    annotated_res <- merge(res_data, gene_name, by.x = "V1", by.y = "ensembl_gene_id", all.x = TRUE)
    annotated_file <- file.path(output_path, paste0(result_name, "_annotated.csv"))
    write.csv(annotated_res, annotated_file, row.names = FALSE)
    file.remove(raw_res_file)
  }
}

run_differential_expression <- function(txi_salmon, samples_file, gene_name_file, output_path, ref_group) {
  samples <- load_samples(samples_file)
  gene_name <- load_gene_names(gene_name_file)
  
  dds <- prepare_deseq2(txi_salmon, samples, ref_group)

  dds <- DESeq(dds)
  
  # Perform PCA and save plots
  perform_pca(dds, output_path)
  
  # Process and annotate DESeq2 results
  process_deseq2_results(dds, gene_name, output_path)
}


### Step 0: Define directory

In [ ]:
samples_file = "/hdd1/projects/rand/GLP1/GSE218026/sample.txt"
salmon_path = "/hdd1/projects/rand/GLP1/GSE218026/"
output_path = "/hdd1/projects/rand/GLP1/GSE218026/test_script/"
organism_type = "mouse" #"human"
ref_group = "Sham"

### Step 1: Generate TPM Matrix

In [ ]:
if (organism_type == "mouse") {
    gene_name_file = "/hdd1/projects/bulk_expression/symbol_to_gene/mouse_ensmus_to_geneName.csv"
    tx2gene_file = "/hdd1/projects/bulk_expression/symbol_to_gene/ENSMUST_to_ENSMUSG.csv"
} else if(organism_type == "human") {
    gene_name_file = "/hdd1/projects/bulk_expression/symbol_to_gene/biomart_hg38_ensg_geneName.csv"
    tx2gene_file = "/hdd1/projects/bulk_expression/symbol_to_gene/enst_to_ensg.csv"
}
#Run function
txi_output <- create_tpm_matrix(
  samples_file,
  gene_name_file,
  salmon_path,
  tx2gene_file,
  output_path
)

### Step 2: Differential Expression Analysis

In [ ]:
if (organism_type == "mouse") {
    gene_name_file = "/hdd1/projects/bulk_expression/symbol_to_gene/mouse_ensmus_to_geneName.csv"
} else if(organism_type == "human") {
    gene_name_file = "/hdd1/projects/bulk_expression/symbol_to_gene/biomart_hg38_ensg_geneName.csv"
}

#Run function
run_differential_expression(
  txi_salmon = txi_output,
  samples_file,
  gene_name_file,
  output_path,
  ref_group
)